# <center>Trabajo práctico N°4</center>

In [2]:
using LinearAlgebra
using Plots
# using SymPy

In [46]:
function get_nodes_ix(nodes, nelem)
    ix = zeros(nodes)
    for i in 1:nodes
        ix[i] = (nodes - 1) * (nelem - 1) + i
    end
    return map(y -> round(Int,y), ix)  # to avoid float indices.
end

get_nodes_ix (generic function with 1 method)

In [47]:
function simpson_rule_l(i, j, ϕ_prime)
    q = [0 1/2 1]
    weights = [1/6 4/6 1/6]
    rv = 0
    for k in 1:3
        rv = rv + ϕ_prime[i](q[k]) * ϕ_prime[j](q[k]) * weights[k]
    end
    return rv
end

simpson_rule_l (generic function with 2 methods)

In [49]:
function simpson_rule_f(k, fun, h)
    return fun(h*(k-1) + h/2) * (h/2)
end

simpson_rule_f (generic function with 2 methods)

In [61]:
function right_hand_side(i, fun, ϕ, total_nodes, h)
    rv = Array{Float64}(undef, total_nodes)
    for j in 1:total_nodes
        rv[j] = simpson_rule_f(h*(j-1), h, fun, ϕ)
    end
    return rv
end

right_hand_side (generic function with 1 method)

In [62]:
function finite_elements(nelem, fun, ϕ, ϕ_prime; method="linear")
    @assert (method in ["linear", "quad"])
    h = 1/nelem
    nodes_by_element = 2
    total_nodes = nelem*(nodes_by_element-1) + 1
    F = zeros(nelem + 1)
    # local matrix
    local_matrix = zeros(nodes_by_element, nodes_by_element)
    for i in 1:nodes_by_element, j in 1:nodes_by_element
        local_matrix[i, j] = simpson_rule_l(i, j, ϕ_prime)
    end

    # global matrix
    global_matrix = zeros(nelem+1, nelem+1)
    for i in 1:nelem
        nodes_ix = get_nodes_ix(nodes_by_element, i)
        i1, i2 = nodes_ix[1], nodes_ix[2]
        global_matrix[i1:i2, i1:i2] = global_matrix[i1:i2, i1:i2] + local_matrix
        
        # right-hand side
        F[i] = F[i] .+ right_hand_side(i, fun, ϕ, total_nodes, h)
    end
    F[1], F[end] = 0, 0
    global_matrix = global_matrix .* h
    
    # border conditions
    global_matrix[2:end, 1] .= 0
    global_matrix[1, 2:end] .= 0
    global_matrix[1, 1] = 1
    
    global_matrix[1:end-1, end] .= 0
    global_matrix[end, 1:end-1] .= 0
    global_matrix[end, end] = 1
    return global_matrix
end

finite_elements (generic function with 3 methods)

In [63]:
ϕ_1 = x -> x
ϕ_2 = x -> 1 - x
ϕ = [ϕ_1 ϕ_2]

ϕ_1_prime = x -> -1
ϕ_2_prime = x -> 1
ϕ_prime = [ϕ_1_prime ϕ_2_prime]

f = x -> 1

finite_elements(4, f, ϕ, ϕ_prime)

LoadError: MethodError: objects of type Matrix{Function} are not callable
Use square brackets [] for indexing an Array.

In [23]:
q = "quad"
q in ["linear", "quad"]

true